In [250]:
import pandas as pd
import json
import re
import numpy as np

#### Olympic Medal Dataset Preparation

In [60]:
# Importing dataset
OlympicsDataset = pd.read_json('olympics_medals_dataset.json')

In [61]:
# Rename columns for dot notation
OlympicsDataset = OlympicsDataset.rename(columns = {'Team':'Country',
'NgamesSOG':'N_games_S',
'Gold SOG':'Gold_S',
'Silver SOG':'Silver_S',
'Bronze SOG':'Bronze_S',
'Total SOG':'Total_S',
'NgamesWOG':'N_games_W',
'Gold WOG':'Gold_W',
'Silver WOG':'Silver_W',
'Bronze WOG':'Bronze_W',
'Total WOG':'Total_W',
'NgamesT':'N_games_C',
'Gold T':'Gold_C',
'Silver T':'Silver_C',
'Bronze T':'Bronze_C',
'Total T':'Total_C'})

In [64]:
# String replacements
OlympicsDataset = OlympicsDataset.replace('\n','', regex=True)
OlympicsDataset = OlympicsDataset.replace(',','', regex=True)

In [71]:
OlympicsDataset.Gold_S = OlympicsDataset.Gold_S.astype(int)
OlympicsDataset.Total_S = OlympicsDataset.Total_S.astype(int)
OlympicsDataset.Gold_C = OlympicsDataset.Gold_C.astype(int)

In [125]:
# Find problems in Total_C column and removes them
listChange = []
for index,line in enumerate(OlympicsDataset.Total_C):
    match = re.search(r'\[(.*?)\]',line)
    if match != None:
        listChange.append(index)
        OlympicsDataset.Total_C[index] = re.sub(r'\[(.*?)\]','',line)

<ipython-input-125-b5cafc3f738f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.Total_C[index] = re.sub(r'\[(.*?)\]','',line)


43    888
66    742
Name: Total_C, dtype: object

In [130]:
OlympicsDataset.Total_C = OlympicsDataset.Total_C.astype(int)

In [135]:
# Find problems in Country column and removes them
listChange = []
for index,line in enumerate(OlympicsDataset.Country):
    match = re.search(r'\[(.*?)\]',line)
    if match != None:
        listChange.append(index)
        OlympicsDataset.Country[index] = re.sub(r'\[(.*?)\]','',line)

<ipython-input-135-f342bec20471>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.Country[index] = re.sub(r'\[(.*?)\]','',line)


4        12
5       562
10        1
14        4
17        2
18      230
23       13
24      696
27        4
29      241
31       98
32      168
33      206
34        1
37       38
43      888
46      892
47      137
48      519
49      243
50        5
51      948
55        1
56        2
57        9
60       35
62       76
66      742
67       87
81        4
82       13
91      451
92        1
93      140
97      528
101       4
108     547
109       8
110    1204
111     135
112      17
118      24
119       2
121      40
124     171
125       2
127       2
128     661
131      36
133       2
137      19
144    2963
150      94
151       2
152       8
153       2
154       3
155      16
Name: Total_C, dtype: int32

In [172]:
# Separating country TAG
OlympicsDataset.Country = OlympicsDataset.Country.str.replace('\xa0',' ').str.lstrip()

In [ ]:
result = OlympicsDataset.Country.str.extract(r'(.*)(\s)\((.*?)\)')

In [184]:
result.drop(1,axis=1,inplace=True)

In [202]:
result = result.rename(columns={0:'Country_name',2:'Tag'})

In [204]:
result2 = pd.concat([OlympicsDataset,result],axis=1)

,Country,N_games_S,Gold_S,Silver_S,Bronze_S,Total_S,N_games_W,Gold_W,Silver_W,Bronze_W,Total_W,N_games_C,Gold_C,Silver_C,Bronze_C,Total_C,Country_name,Tag
0,Afghanistan (AFG),15,0,0,2,2,0,0,0,0,0,15,0,0,2,2,Afghanistan,AFG
1,Algeria (ALG),14,5,4,8,17,3,0,0,0,0,17,5,4,8,17,Algeria,ALG
2,Argentina (ARG),25,21,26,30,77,19,0,0,0,0,44,21,26,30,77,Argentina,ARG
3,Armenia (ARM),7,2,8,8,18,7,0,0,0,0,14,2,8,8,18,Armenia,ARM
4,Australasia (ANZ),2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,Australasia,ANZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Zambia (ZAM),14,0,1,1,2,0,0,0,0,0,14,0,1,1,2,Zambia,ZAM
152,Zimbabwe (ZIM),14,3,4,1,8,1,0,0,0,0,15,3,4,1,8,Zimbabwe,ZIM
153,Independent Olympic Athletes (IOA),3,1,0,1,2,0,0,0,0,0,3,1,0,1,2,Independent Olympic Athletes,IOA
154,Independent Olympic Participants (IOP),1,0,1,2,3,0,0,0,0,0,1,0,1,2,3,Independent Olympic Participants,IOP


In [205]:
result2.drop('Country',axis=1,inplace=True)

In [298]:
OlympicsDataset = result2.reindex(['Country_name','Tag','N_games_S', 'Gold_S', 'Silver_S', 'Bronze_S', 'Total_S', 'N_games_W',
       'Gold_W', 'Silver_W', 'Bronze_W', 'Total_W', 'N_games_C', 'Gold_C',
       'Silver_C', 'Bronze_C', 'Total_C'],axis=1)

In [299]:
OlympicsDataset.head()

,Country_name,Tag,N_games_S,Gold_S,Silver_S,Bronze_S,Total_S,N_games_W,Gold_W,Silver_W,Bronze_W,Total_W,N_games_C,Gold_C,Silver_C,Bronze_C,Total_C
0,Afghanistan,AFG,15,0,0,2,2,0,0,0,0,0,15,0,0,2,2
1,Algeria,ALG,14,5,4,8,17,3,0,0,0,0,17,5,4,8,17
2,Argentina,ARG,25,21,26,30,77,19,0,0,0,0,44,21,26,30,77
3,Armenia,ARM,7,2,8,8,18,7,0,0,0,0,14,2,8,8,18
4,Australasia,ANZ,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


#### Country wealth dataset preparation

In [228]:
# Importing dataset
WealthDataset = pd.read_json('country_wealth_dataset.json')

In [215]:
WealthDataset.head()

,Country,Median,Mean,Gini,Adults
0,Luxembourg\n,"259,899\n","477,306\n",67.0,"498,000\n"
1,Australia\n,"238,072\n","483,755\n",65.6,"19,159,000\n"
2,Iceland\n,"231,462\n","337,787\n",50.9,"255,000\n"
3,Belgium\n,"230,548\n","351,327\n",60.3,"8,993,000\n"
4,Hong Kong\n,"173,768\n","503,335\n",74.6,"6,292,000\n"


In [216]:
WealthDataset.dtypes

Country     object
Median      object
Mean        object
Gini       float64
Adults      object
dtype: object

In [320]:
WealthDataset = WealthDataset.replace('\n','', regex=True).replace(',','',regex=True).replace('\xa0','',regex=True)

In [230]:
WealthDataset.Median = WealthDataset.Median.astype(int)
WealthDataset.Mean = WealthDataset.Mean.astype(int)
WealthDataset.Adults = WealthDataset.Adults.astype(int)

In [321]:
WealthDataset.head()

,Country,Median,Mean,Gini,Adults
0,Luxembourg,259899,477306,67.0,498000
1,Australia,238072,483755,65.6,19159000
2,Iceland,231462,337787,50.9,255000
3,Belgium,230548,351327,60.3,8993000
4,Hong Kong,173768,503335,74.6,6292000


#### Country education dataset preparation

In [292]:
# Importing dataset
EducationDataset = pd.read_json('country_education_dataset.json')

In [293]:
EducationDataset.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.122,0.133,0.145,0.156,0.168,0.179,0.19,0.202,0.213,...,0.372,0.374,0.39,0.398,0.403,0.405,0.406,0.408,0.413,0.414
1,Albania,0.583,0.588,0.557,0.542,0.541,0.55,0.557,0.569,0.579,...,0.671,0.714,0.739,0.749,0.758,0.753,0.745,0.747,0.743,0.746
2,Algeria,0.385,0.395,0.405,0.414,0.424,0.431,0.443,0.458,0.473,...,0.626,0.644,0.639,0.639,0.652,0.659,0.66,0.665,0.668,0.672
3,Andorra,,,,,,,,,,...,0.67,0.671,0.724,0.714,0.725,0.718,0.722,0.713,0.720,0.720
4,Angola,,,,,,,,,,...,0.398,0.423,0.435,0.447,0.46,0.472,0.487,0.498,0.500,0.500


In [311]:
EducationDataset = EducationDataset.replace(r'^\s*$', np.nan, regex=True).replace('\xa0','',regex=True)

In [312]:
columnsList = EducationDataset.columns
EducationDataset[columnsList[1:]] = EducationDataset[columnsList[1:]].astype(float)

In [313]:
EducationDataset.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.122,0.133,0.145,0.156,0.168,0.179,0.190,0.202,0.213,...,0.372,0.374,0.390,0.398,0.403,0.405,0.406,0.408,0.413,0.414
1,Albania,0.583,0.588,0.557,0.542,0.541,0.550,0.557,0.569,0.579,...,0.671,0.714,0.739,0.749,0.758,0.753,0.745,0.747,0.743,0.746
2,Algeria,0.385,0.395,0.405,0.414,0.424,0.431,0.443,0.458,0.473,...,0.626,0.644,0.639,0.639,0.652,0.659,0.660,0.665,0.668,0.672
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.670,0.671,0.724,0.714,0.725,0.718,0.722,0.713,0.720,0.720
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.398,0.423,0.435,0.447,0.460,0.472,0.487,0.498,0.500,0.500


#### Data export

In [323]:
EducationDataset.to_csv(r'D:\Learning\EstudosDataScience\Projetos\DataScienceProjects\20210826 Olympics Analysis\3 - Dados de Upload\EducationDataset.csv', index=False,)

In [325]:
WealthDataset.to_csv(r'D:\Learning\EstudosDataScience\Projetos\DataScienceProjects\20210826 Olympics Analysis\3 - Dados de Upload\WealthDataset.csv', index=False,)

In [326]:
OlympicsDataset.to_csv(r'D:\Learning\EstudosDataScience\Projetos\DataScienceProjects\20210826 Olympics Analysis\3 - Dados de Upload\OlympicsDataset.csv', index=False,)